## 最終課題

In [3]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, urlunparse, parse_qsl, urlencode
import time

base_url = 'https://www.musashino-u.ac.jp/'
domain = 'musashino-u.ac.jp'

visited_urls = set()
urls_to_visit = [base_url]
url_title_dict = {}

# カウンター初期化
visited_count = 0
total_found = 1  # 最初のURLを含む

# 除外するパス（portal配下など）
excluded_paths = [
    '/portal/',  # 必要に応じて追加
]

# 除去するクエリパラメータ（追跡/動的）
drop_query_keys = {
    'utm_source','utm_medium','utm_campaign','utm_term','utm_content',
    'gclid','fbclid','mc_cid','mc_eid','yclid'
}

def normalize_url(current, href):
    # 絶対URL化
    abs_url = urljoin(current, href)

    # URLを構成要素に分解
    parsed = urlparse(abs_url)
    
    # path に ;param が含まれるケースを除去
    path = parsed.path.split(';')[0]  # ;以降を落とす

    # クエリパラメータの処理 - 簡素化バージョン
    # 1. クエリを取得してキーと値のペアに分解
    query_items = parse_qsl(parsed.query, keep_blank_values=True)
    
    # 2. 不要なパラメータを除外
    filtered_query = []
    for key, value in query_items:
        if key not in drop_query_keys:
            filtered_query.append((key, value))
    
    # 3. パラメータをソート
    filtered_query.sort()
    
    # 4. 整形したクエリ文字列に変換
    norm_query = urlencode(filtered_query, doseq=True)

    # フラグメントは落とす
    normalized = urlunparse((parsed.scheme, parsed.netloc, path, '', norm_query, ''))
    return normalized

def is_same_domain(url):
    return urlparse(url).netloc.endswith(domain)

def is_excluded(url):
    p = urlparse(url).path
    return any(p.startswith(excl) for excl in excluded_paths)

def is_asset(url):
    lower = url.lower()
    return lower.endswith((
        '.pdf','.jpg','.jpeg','.png','.gif','.zip','.rar','.7z',
        '.mp3','.mp4','.mov','.avi','.wmv','.mkv',
        '.doc','.docx','.xls','.xlsx','.ppt','.pptx','.csv',
        '.svg','.webp','.ico'
    ))

session = requests.Session()
session.headers.update({
    'User-Agent': 'SimpleCrawler/1.0 (+for study; [mail])'
})

while urls_to_visit:
    current_url = urls_to_visit.pop(0)
    if current_url in visited_urls:
        continue
    
    visited_urls.add(current_url)
    visited_count += 1

    print(f"訪問中 [{visited_count}/{total_found}]: {current_url}")
    time.sleep(0.25)

    try:
        resp = session.get(current_url, timeout=10, allow_redirects=True)
        if resp.status_code != 200:
            continue

        # HTMLのみ処理
        ctype = resp.headers.get('Content-Type', '')
        if 'text/html' not in ctype:
            continue

        resp.encoding = resp.apparent_encoding
        soup = BeautifulSoup(resp.text, 'html.parser')

        title_tag = soup.find('title')
        title = title_tag.get_text(strip=True) if title_tag else 'タイトルなし'
        url_title_dict[current_url] = title

        for a in soup.find_all('a', href=True):
            if a.get('rel') and 'nofollow' in [r.lower() for r in a['rel']]:
                continue
            cand = normalize_url(current_url, a['href'])
            if not is_same_domain(cand):
                continue
            if is_excluded(cand):
                continue
            if is_asset(cand):
                continue
            if cand not in visited_urls and cand not in urls_to_visit:
                urls_to_visit.append(cand)
                total_found += 1

    except Exception as e:
        print(f"エラー: {current_url} - {e}")

# 結果を表示
print(f"\n訪問完了 - 訪問したページ数: {visited_count}")
print(f"見つかったURL総数: {total_found}")
print(f"収集したタイトル数: {len(url_title_dict)}")
print("\nURL-タイトル辞書:")
print(url_title_dict)

訪問中 [1/1]: https://www.musashino-u.ac.jp/
訪問中 [2/105]: https://ef.musashino-u.ac.jp/donation/
訪問中 [3/127]: https://www.musashino-u.ac.jp/access.html
訪問中 [4/135]: https://www.musashino-u.ac.jp/admission/request.html
訪問中 [5/169]: https://www.musashino-u.ac.jp/contact.html
訪問中 [6/171]: https://www.musashino-u.ac.jp/prospective-students.html
訪問中 [7/183]: https://www.musashino-u.ac.jp/students.html
訪問中 [8/191]: https://www.musashino-u.ac.jp/alumni.html
訪問中 [9/193]: https://www.musashino-u.ac.jp/parents.html
訪問中 [10/195]: https://www.musashino-u.ac.jp/business.html
訪問中 [11/197]: https://www.musashino-u.ac.jp/guide/
訪問中 [12/218]: https://www.musashino-u.ac.jp/guide/profile/
訪問中 [13/243]: https://www.musashino-u.ac.jp/guide/activities/
訪問中 [14/245]: https://www.musashino-u.ac.jp/guide/campus/
訪問中 [15/246]: https://www.musashino-u.ac.jp/guide/facility/
訪問中 [16/279]: https://www.musashino-u.ac.jp/guide/information/
訪問中 [17/311]: https://www.musashino-u.ac.jp/guide/profile/media/
訪問中 [18/312]: ht